In [1]:
# Import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [2]:
# Setup Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url = "https://redplanetscience.com/"
browser.visit(url)

In [3]:
# Confirm the HTML element to isolate the title and preview is present within the browser.
browser.is_element_present_by_css("div.list_text", 1)

True

In [4]:
# Create variables for browser.html to use as a Beautiful Soup object and parse out the webpage
mars = browser.html
mars_soup = bs(mars, "html.parser")

In [5]:
# Use the soup object to find the results for the HTML element
mars_results = mars_soup.find_all("div", class_="list_text")

In [6]:
# Create an empty list to append the individual dictionaries to.
mars_dict_list = []

In [7]:
# Connect to MongoDB and create a database to export results
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.mars_db

# Declare the collection
articles = db.articles

In [19]:
# Loop through the mars_results and find the title and preview, convert to text, create a dictionary for the
# result and append the dictionary to the empty list.
for result in mars_results:
    title = result.find("div", class_="content_title").text
    preview = result.find("div", class_="article_teaser_body").text
    mars_dict = {"title": title, "preview": preview}
    mars_dict_list.append(mars_dict)
    articles.insert_one(mars_dict)
    print(f'''Article Title: {title}
Summary: {preview}
----------------------------------------''')

Article Title: The Man Who Wanted to Fly on Mars
Summary: The Mars Helicopter is riding to the Red Planet this summer with NASA's Perseverance rover. The helicopter's chief engineer, Bob Balaram, shares the saga of how it came into being.
----------------------------------------
Article Title: Mars Is Getting a New Robotic Meteorologist
Summary: Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.
----------------------------------------
Article Title: 5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover
Summary: The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.
----------------------------------------
Article Title: Mars 2020 Unwrapped and Ready for More Testing
Summary: In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for t

In [9]:
# Quit the browser session.
browser.quit()

In [12]:
# View the data in the list of dictionaries
mars_dict_list

[{'title': 'The Man Who Wanted to Fly on Mars',
  'preview': "The Mars Helicopter is riding to the Red Planet this summer with NASA's Perseverance rover. The helicopter's chief engineer, Bob Balaram, shares the saga of how it came into being.",
  '_id': ObjectId('631f752fd91283c0ff8ac824')},
 {'title': 'Mars Is Getting a New Robotic Meteorologist',
  'preview': "Sensors on NASA's Perseverance will help prepare for future human exploration by taking weather measurements and studying dust particles.",
  '_id': ObjectId('631f7530d91283c0ff8ac825')},
 {'title': "5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover",
  'preview': "The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.",
  '_id': ObjectId('631f7530d91283c0ff8ac826')},
 {'title': 'Mars 2020 Unwrapped and Ready for More Testing',
  'preview': "In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's M